In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn_cell
from tensorflow.contrib import rnn
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

hm_epochs = 20
n_classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28
rnn_size = 128


x = tf.placeholder('float', [None, n_chunks,chunk_size])
y = tf.placeholder('float')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
def recurrent_neural_network(x):
    
    layer = {'weights': tf.Variable(tf.truncated_normal([rnn_size, n_classes], stddev=0.1)), 'biases': tf.Variable(tf.constant(0.1, shape=[n_classes]))}

    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)

    lstm_cell = rnn_cell.BasicLSTMCell(rnn_size, state_is_tuple=True)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    output = tf.matmul(outputs[-1],layer['weights']) + layer['biases']

    return output

In [0]:
def train_neural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y))
    
    # minimize cost
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
    
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))

                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels})*100)


In [4]:
# Run
train_neural_network(x)

Epoch 1 completed out of 20 loss: 240.69318786263466
Epoch 2 completed out of 20 loss: 70.46602479368448
Epoch 3 completed out of 20 loss: 48.40774934180081
Epoch 4 completed out of 20 loss: 36.97551094181836
Epoch 5 completed out of 20 loss: 28.967577564995736
Epoch 6 completed out of 20 loss: 24.979740807320923
Epoch 7 completed out of 20 loss: 20.301043623127043
Epoch 8 completed out of 20 loss: 17.917350572068244
Epoch 9 completed out of 20 loss: 16.620792207075283
Epoch 10 completed out of 20 loss: 14.164775706012733
Epoch 11 completed out of 20 loss: 13.543566430686042
Epoch 12 completed out of 20 loss: 11.330714250681922
Epoch 13 completed out of 20 loss: 10.71738048212137
Epoch 14 completed out of 20 loss: 10.11939733615145
Epoch 15 completed out of 20 loss: 9.893829475506209
Epoch 16 completed out of 20 loss: 8.304935497464612
Epoch 17 completed out of 20 loss: 8.245884006784763
Epoch 18 completed out of 20 loss: 7.086872466374189
Epoch 19 completed out of 20 loss: 6.639146919